# DLA for Z2 lattice gauge theory Hamiltonian variational ansatz (dense implementation)

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import h5py
from qiskit.quantum_info import SparsePauliOp
from fastdla.dla_dense import generate_dla, is_independent
from fastdla.ansatze.z2lgt_hva import z2lgt_hva_generators, z2lgt_dense_projector
import jax
jax.config.update('jax_enable_x64', True)
import jax.numpy as jnp


In [2]:
data_file = 'z2lgt_hva_dla_dense.h5'

## 2 Fermions

In [3]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
charge = 0

In [4]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full


[SparsePauliVector(['IIIIIIIZ', 'IIIZIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIZII', 'IZIIIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIIXI', 'IIIIXIII', 'IIXIIIII', 'XIIIIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIIIXZX', 'IIIIIYZY', 'IXZXIIII', 'IYZYIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIXZXII', 'IIIYZYII', 'ZXIIIIIX', 'ZYIIIIIY'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j]))]

In [5]:
# Projector
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge, npmod=jnp)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')
csrs = [SparsePauliOp(op.paulis, op.coeffs).to_matrix(sparse=True) for op in generators_full]
generators = [dense_projector @ mat.dot(dense_projector.conjugate().T) for mat in csrs]
generators = [op / np.sqrt(np.abs(np.trace(op.conjugate().T @ op))) for op in generators]

Hilbert subspace dimension is 12


In [6]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [7]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

Basis size 64; 2000th commutator [b[62], b[47]]
Found 60 new ops in 1.19s
Subspace DLA dimension is 64


In [8]:
with h5py.File(data_file, 'a') as out:
    if 'nf=2' not in out:
        nf_group = out.create_group('nf=2')
        for gname, oplist in [
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('ops', data=np.array(oplist))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)

## 3 Fermions

In [9]:
num_fermions = 3
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1]
charge = 0

In [10]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)
generators_full

[SparsePauliVector(['IIIIIIIIIIIZ', 'IIIIIIIZIIII', 'IIIZIIIIIIII'], array([0.57735027+0.j, 0.57735027+0.j, 0.57735027+0.j])),
 SparsePauliVector(['IIIIIIIIIZII', 'IIIIIZIIIIII', 'IZIIIIIIIIII'], array([0.57735027+0.j, 0.57735027+0.j, 0.57735027+0.j])),
 SparsePauliVector(['IIIIIIIIIIXI', 'IIIIIIIIXIII', 'IIIIIIXIIIII', 'IIIIXIIIIIII', 'IIXIIIIIIIII', 'XIIIIIIIIIII'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j])),
 SparsePauliVector(['IIIIIIIIIXZX', 'IIIIIIIIIYZY', 'IIIIIXZXIIII', 'IIIIIYZYIIII', 'IXZXIIIIIIII', 'IYZYIIIIIIII'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j])),
 SparsePauliVector(['IIIIIIIXZXII', 'IIIIIIIYZYII', 'IIIXZXIIIIII', 'IIIYZYIIIIII', 'ZXIIIIIIIIIX', 'ZYIIIIIIIIIY'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j]))]

In [11]:
# Projector
dense_projector = z2lgt_dense_projector(gauss_eigvals, charge)
print(f'Hilbert subspace dimension is {dense_projector.shape[0]}')
csrs = [SparsePauliOp(op.paulis, op.coeffs).to_matrix(sparse=True) for op in generators_full]
generators = [dense_projector @ mat.dot(dense_projector.conjugate().T) for mat in csrs]
generators = [op / np.sqrt(np.abs(np.trace(op.conjugate().T @ op))) for op in generators]

Hilbert subspace dimension is 40


In [12]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [13]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=2)
print(f'Subspace DLA dimension is {len(dla)}')

Found 252 new ops in 1.77s
Resizing basis array to 512
Basis size 370; 2000th commutator [b[62], b[47]]
Basis size 394; 4000th commutator [b[88], b[84]]
Basis size 398; 6000th commutator [b[109], b[5]]
Basis size 398; 10000th commutator [b[140], b[130]]
Basis size 398; 12000th commutator [b[154], b[65]]
Basis size 398; 14000th commutator [b[166], b[139]]
Basis size 398; 16000th commutator [b[178], b[69]]
Basis size 398; 18000th commutator [b[189], b[45]]
Basis size 398; 20000th commutator [b[199], b[100]]
Basis size 398; 22000th commutator [b[209], b[55]]
Basis size 398; 24000th commutator [b[218], b[129]]
Basis size 398; 26000th commutator [b[227], b[122]]
Basis size 398; 28000th commutator [b[236], b[34]]
Basis size 398; 30000th commutator [b[244], b[110]]
Basis size 398; 32000th commutator [b[252], b[122]]
Basis size 398; 34000th commutator [b[260], b[70]]
Basis size 398; 36000th commutator [b[267], b[222]]
Basis size 398; 38000th commutator [b[275], b[50]]
Basis size 398; 40000th c

In [14]:
with h5py.File(data_file, 'a') as out:
    if 'nf=3' not in out:
        nf_group = out.create_group('nf=3')
        for gname, oplist in [
            ('generators_symm0', generators_indep),
            ('dla_symm0', dla)
        ]:
            group = nf_group.create_group(gname)
            group.create_dataset('ops', data=np.array(oplist))

        nf_group.create_dataset('dla_symm0/gauss_eigvals', data=gauss_eigvals)
        nf_group.create_dataset('dla_symm0/charge', data=charge)